In [206]:
from sympy import *
from sympy.physics.units import *
import pint
ureg = pint.UnitRegistry(auto_reduce_dimensions=True)
import re
from unyt import N, m, s, kg, mm, MPa, unyt_quantity

In [207]:
prefixes = [
    "yocto",
    "zepto",
    "atto",
    "femto",
    "pico",
    "nano",
    "micro",
    "milli",
    "centi",
    "deci",
    "deca",
    "hecto",
    "kilo",
    "mega",
    "giga",
    "tera",
    "peta",
    "exa",
    "zetta",
    "yotta",
]

baseUnits = [
    "newton",
    "meter",
    "joule"
]

def createUnitsDict(prefixes, baseUnits):
    dict = {}
    for u in baseUnits:
        dict[u] = globals()[u]
        for p in prefixes:
            dict[p + u] = globals()[p].scale_factor * globals()[u]
    return dict

unitDict = createUnitsDict(prefixes, baseUnits)

def symparse(expStr):
    return parse_expr(expStr, local_dict=unitDict, evaluate=False)

def parseParamStr(string):
    tokens = string.split('=')
    return list(map(lambda s: s.strip(), tokens))

def parseInlineExp(expStr):
    return re.findall(r'\[(.*?)\]', expStr)

def convertToPint(inlineParams):
    return list(map(lambda p: ureg(p), inlineParams))

def convertToSym(matchObj):
    wholeStr = matchObj.group(0)
    unitStr = matchObj.group(1)
    return wholeStr.replace(unitStr, "* (" + str(ureg(unitStr).u) + ")").replace("[", "").replace("]", "")

In [208]:
paramStr = "a = [1000000 millinewton] = m = parameterA"
paramTokens = parseParamStr(paramStr)
inlineParams = convertToPint(parseInlineExp(paramTokens[1]))
inlineUnits = list(map(lambda p: str(p.u), inlineParams))

exx = re.sub(r'\[.*?\s(.*?)\]', convertToSym, paramTokens[1])
exp = symparse(exx).simplify()
print(exx)
print(symparse(exx))
a = ureg(str(exp))
b = unyt_quantity.from_pint(a)
print(str(b))

1000000 * (millinewton)
1000000*(newton/1000)
1000 N
